In [1]:
import sys
sys.path.insert(0, '../')

from src.time_tagger import TimeTaggerController, TT_Simulator, two_particle_states
from src.kinetic_mount_controller import KineticMountControl
from src.osc import OSCCommunicator, OSCTarget
import time
import threading
import random

In [2]:
# only run if not already created
if 'KMC' not in globals():  
    KMC = KineticMountControl(number_of_devices=2, address_search_depth=0)
    #KMC.swap_alice_bob()

TTC = TimeTaggerController()
# connect KMC and TTC
TTC.setKineticMountController(KMC)
# set alice channels
TTC.set_alice_reflection_channel(1)
TTC.set_alice_transmission_channel(2)
# set bob channels
TTC.set_bob_reflection_channel(4)
TTC.set_bob_transmission_channel(3)


# set bob home offset
KMC.bob.set_home_offset(-43 % 360)
KMC.bob.home()
# set alice home offset
KMC.alice.set_home_offset(9)
KMC.alice.home()




ASSIGNING CONTROLLERS
---------------------------------------------------------------
Ports Found: ['COM8', 'COM9']
---------------------------------------------------------------
Device succesfully connected (Rotation Mount) (Address: 0)
Device succesfully connected (Rotation Mount) (Address: 0)


In [3]:
_ = TTC.performDelayAdjustment(integration_time=5)#, manual_delays=[0, 947, 78, -502])

Delays Before Correction
Alice_T :     0 ps 	/     0.0 mm
Alice_R :   -26 ps 	/    -7.8 mm
Bob_T   :  -551 ps 	/  -165.3 mm
Bob_R   :  -246 ps 	/   -73.8 mm

Delays After Correction
Alice_T :     0 ps 	/     0.0 mm
Alice_R :    10 ps 	/     3.0 mm
Bob_T   :     4 ps 	/     1.2 mm
Bob_R   :   -38 ps 	/   -11.4 mm


In [4]:
n_values = 150
trace_width = 3
TTC.displayCountTraces(n_values=n_values, trace_width=trace_width, plot_title='<b>Raw Counts<b>')
TTC.displayCoincidenceTraces(coincidence_window_SI=0.5e-9, n_values=n_values, trace_width=trace_width, plot_title='<b>Coincidences<b>')

Output()

Output()

No such comm: 7f4b49f4d4b444bbb91b6abdc7be9710
No such comm: b6ab7f7c1d584c6aa773c7e865897881


In [3]:
KMC.rotate_simulataneously(0, 0, wait_for_completion=False)

In [4]:
# Global flag to control the loop
resultTranslate = {0 : [1, 1],
                   1 : [1, -1],
                   2 : [-1, 1],
                   3 : [-1, -1]}
# a A b B
Q_angles = [0, -22.5, 11.5, -11.5]
C_angles = [0, 22.5, -45, -22.5]

                    # ab, aB, Ab, AB
angle_pair_dict = {'Q_all':[(0, 11.5), (0, -11.5), (-22.5, 11.5), (-22.5, -11.5)],
                    # ab, AB
                   'Q_diagonal': [(0, 11.5), (-22.5, -11.5)],
                    # ab, AB
                   'C_diagonal': [(0, -45), (22.5, -22.5)],
                    # AB
                   'C_single': [(-45, -22.5)],
                   }

# for this perf we only want Q_all
angle_pairs = angle_pair_dict['Q_all']

# setup system with correct initial angles
KMC.rotate_simulataneously(*angle_pairs[0])

def performance_cycle(cycles):
    """The function you want to execute repeatedly."""
   
    time_of_last_visual = time.time()
    prev_theta_a, prev_theta_b = angle_pairs[0]
    for _ in range(cycles):
        theta_a, theta_b = random.choice(angle_pairs)
        
        measurement_results, prev_theta_a, prev_theta_b = TTC.get_single_measurement_metronome(angle_pairs, theta_a, theta_b, prev_theta_a, prev_theta_b, integration_time=0.065, max_integration_time=0.07, max_rotation_duration=0.35)
        

In [6]:
TTC.createCoincidenceChannels(0.5e-9)

In [4]:
start = time.perf_counter()
result = TTC.collect_stream_data_single_attempt(0.065, 0.065)
print(time.perf_counter() - start, result )

AttributeError: 'NoneType' object has no attribute 'getChannels'

In [5]:
performance_cycle(100)

Timing Summary: {'pre_rotation_time': 0.07006900000851601, 'rotate_simultaneously': 0.3501082000148017, 'post_rotation_time': 0.07002610000199638, 'total': 0.49054600001545623}
Timing Summary: {'pre_rotation_time': 0.07008960001985542, 'rotate_simultaneously': 0.3500621000130195, 'post_rotation_time': 0.0700467000133358, 'total': 0.49020519998157397}
Timing Summary: {'pre_rotation_time': 0.07005050001316704, 'rotate_simultaneously': 0.35009849999914877, 'post_rotation_time': 0.07005770000978373, 'total': 0.49021439999341965}
Timing Summary: {'pre_rotation_time': 0.07004689998575486, 'rotate_simultaneously': 0.3500515000196174, 'post_rotation_time': 0.07003150001401082, 'total': 0.4901370999868959}
Timing Summary: {'pre_rotation_time': 0.07004730001790449, 'rotate_simultaneously': 0.35004719998687506, 'post_rotation_time': 0.07037629999103956, 'total': 0.49047749998862855}
Timing Summary: {'pre_rotation_time': 0.0700426000112202, 'rotate_simultaneously': 0.35699210001621395, 'post_rotat

KeyboardInterrupt: 

In [8]:
start = time.perf_counter()
KMC.rotate_simulataneously_metronome(0, 45, wait_for_completion=False,target_duration=0.33)
KMC.rotate_simulataneously_metronome(0, 0, wait_for_completion=False, target_duration=0.33)
KMC.rotate_simulataneously_metronome(0, 45, wait_for_completion=False, target_duration=0.33)
KMC.rotate_simulataneously_metronome(0, 0, wait_for_completion=False, target_duration=0.33)
duration = time.perf_counter() - start
print(duration/4)

0.3302461499988567


# TODO

I will implement the double buffer and make sure they can achieve metronomic clicks by correcting the timing. I might need to pass timing data from previous to next function